In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': ['./Data/mednli_train.csv' ], 'test': './Data/mednli_dev.csv'})


Using custom data configuration default-9bd58d32244e627e
Reusing dataset csv (C:\Users\Subha\.cache\huggingface\datasets\csv\default-9bd58d32244e627e\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence_1', 'sentence_2'],
        num_rows: 11232
    })
    test: Dataset({
        features: ['label', 'sentence_1', 'sentence_2'],
        num_rows: 1395
    })
})

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

In [4]:
dataset["train"][0]

{'label': 'entailment',
 'sentence_1': ' Labs were notable for Cr 1.7 (baseline 0.5 per old records) and lactate 2.4. sentence2: Patient has elevated Cr',
 'sentence_2': ' Patient has elevated Cr'}

In [5]:
model_name = "microsoft/deberta-v3-small"
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 3,id2label = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}, label2id = {'contradiction': 0, 'neutral': 1, 'entailment': 2}) 
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [6]:
metric_name = 'accuracy'

In [7]:
def preprocess_function(examples):
    return tokenizer(examples["sentence_1"], examples["sentence_2"], truncation='True',max_length=128)

encoded_dataset = dataset.map(preprocess_function)

  0%|          | 0/11232 [00:00<?, ?ex/s]

In [59]:
col = dataset["train"].column_names

In [60]:
col.remove('label')
print(col)

['sentence_1', 'sentence_2']


In [61]:
encoded_dataset = encoded_dataset.remove_columns(col)

In [62]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11232
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1395
    })
})

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [63]:
# Finetune the model on dataset for NLI task
from datasets import load_metric

task = "mnli"
metric = load_metric('glue', task)
batch_size = 8
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
   
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
d:\Anaconda\envs\all-purpose-gpu\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11232
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7020


  0%|          | 0/7020 [00:00<?, ?it/s]

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.